## Start, Stop and Delete Amazon EC2 instances

In [1]:
import boto3

ec2 = boto3.client('ec2')

response = ec2.describe_instances()

instance_name = "mlops-prod2"

instance_id="" 

for resp in response['Reservations']:
    resp = resp['Instances'][0]
    tags = resp.get('Tags', [])
    
    for tag in tags:
        if tag.get("Key", "")=="Name" and tag.get("Value", "")==instance_name:
            instance_id = resp['InstanceId']

if instance_id=="":
    print(f"No instance found with name {instance_name}")
    # raise("Stop here!!!")
else:
    print(f"Instance found with name {instance_name}")

print(instance_id)

Instance found with name mlops-prod2
i-03db81b79ad1a797d


In [7]:
# status -> running, stopped, terminated, pending etc.

import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        
        time.sleep(10)


In [4]:
def stop_instances(instance_id):
    print("EC2 Instance Stop")
    ec2.stop_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'stopped')

stop_instances([instance_id])

EC2 Instance Stop
Instance is in stopped state


In [8]:
def start_instances(instance_id):
    print("EC2 Instance Start")
    ec2.start_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'running')

start_instances([instance_id])

response = ec2.describe_instances(InstanceIds=[instance_id])
public_ip = response['Reservations'][0]['Instances'][0]['PublicIpAddress']
public_ip

EC2 Instance Start
Instance is in running state


'54.152.216.108'

# To open SSH Remote:

click on view/command_palette

then type in "Remote-SSH: Connect to Host..."

copy the command in aws browser, instances/connecting/SSH ...

ssh -i /Users/jan/Desktop/advanced_rag/ML_OPS/creds/jani2.pem ubuntu@ec2-54-152-216-108.compute-1.amazonaws.com

unfortunately doesnt work with ssh remote extension from cursor, only works in vs code just typing this in terminal works
public ip changes on restart!!

it is possible to configure an elastic ip to the ec2 instance, so the public ip does not change on restart
this is possible with the aws browser under ec2/elastic ips/allocate elastic ip
then it is necessary to associate the elastic ip with the ec2 instance

# To install packages:

create new conda with conda create -n advanced_rag python=3.11

conda activate advanced_rag

pip install -r clean_requirements.txt


# to configure aws cli

type in terminal: aws configure

access_key, secret_keys are in accessKeys.csv

default region is us-east-1

default output is None

